In [9]:
import sys
sys.path.append('../')

import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

DATA_PATH = "/dust3r/masked_dust3r/data/jackal_training_data_0"

with open(os.path.join(DATA_PATH, "transforms.json"), "r") as f:
    transforms = json.load(f)

In [10]:
tf = []
for f in transforms["frames"]:
    tf.append(np.array(f["transform_matrix"]))
tf = np.array(tf)

#Set z to 0
print("z")
print(tf[:, 2, 3].mean())
print(tf[:, 2, 3].max())
print(tf[:, 2, 3].min())
tf[:, 2, 3] = 0

#Find the euler angles
euler = []
for i in range(tf.shape[0]):
    r = R.from_matrix(tf[i, :3, :3])
    euler.append(r.as_euler("xyz", degrees=True))
euler = np.array(euler)
print("roll")
print(euler[:, 0].mean())
print(euler[:, 0].max())
print(euler[:, 0].min())

if (euler[:,0] > 180).any():
    euler[:,0] -= 360
euler[:,0] = euler[:,0].mean()
#euler[:,0] = -135

print("pitch")
print(euler[:, 1].mean())
print(euler[:, 1].max())
print(euler[:, 1].min())
euler[:, 1] = euler[:,1].mean()

#Convert back to rotation matrix
tf_new = []
for i in range(tf.shape[0]):
    r = R.from_euler("xyz", euler[i], degrees=True)
    tf_new.append(r.as_matrix())
tf_new = np.array(tf_new)

tf[:, :3, :3] = tf_new

for i in range(tf.shape[0]):
    transforms["frames"][i]["transform_matrix"] = tf[i].tolist()

with open(os.path.join(DATA_PATH, "transforms.json"), "w") as f:
    json.dump(transforms, f, indent=4)

z
0.09464387508939455
0.23784152137754427
-0.06390108288485105
roll
-115.88693767155085
179.9526452217328
-179.16341359749518
pitch
1.3326796908128693
43.2089762182229
-43.33539049666404
